# Fashion le Ardeur #

**Group 1**

    Colin Torralba

    Daniel Kier Raluto
    
    Roxanne Bandivas
    
    Czarr Vic Pocol
    
    Jan Rey Viudor


**Description:**

    The very fabric of our society involes the very fabric of our clothes! Clothes, accessories, and etc. help define the culture/lifestyle of the people where unique characteristics/characters come alive that make our society feel alive! In this day and age, it is important to explore and widen our wardrobes, view different fashion senses from other cultures and let it evolve into something new! 

# Project Overview #
    Fashion le Ardeur, is an image classification project that utilizes the Fashion Product Images(Small)(https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small) to create a CNN model that can determine what type of Fashion Product a certain image is.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
from torchvision import transforms 
from torchvision import models

import cv2
import os
import random
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

torch.backends.cudnn.deterministic=True
torch.set_printoptions(sci_mode=False)

from PIL import Image

from tqdm.notebook import tqdm
%matplotlib inline

# Dataset #
    The Dataset was downloaded from kaggle. This is the small version and contains 44,000 images. It also contains a csv file that has columns of descriptions of all 44,000 images.

In [ ]:

# The CSV file is currently in my CWD
desc_file = pd.read_csv('styles.csv')
desc_file['id'] = desc_file['id'].astype(str)


# Opening the data
# The image folder is currently in my laptop storage
# Log 1, after a while of debugging and optimizing, I had to comeback here and fix things, some data were mismatching
image = r'C:\Users\User\Downloads\Elec_4_ass\images'
img_file = os.listdir(image)
img_df = pd.DataFrame({"img_path": img_file})
img_df['id'] = img_df['img_path'].apply(lambda x: os.path.splitext(x)[0])
merged_df = pd.merge(img_df, desc_file, on='id')



### Data Dictionary ###

In [ ]:
# Constructing the data dictionary
data_dict = {'img_path': merged_df['img_path'].apply(lambda x: os.path.join(image, x)).tolist(),
             'type': merged_df['articleType'].tolist()}




In [ ]:
data_dict

In [ ]:
data_dict['img_path'][19]

In [ ]:
data_dict['type'][1]

In [ ]:
Image.open(data_dict['img_path'][1])

So we have constructed the data dictionary with efficiency. We have made a data dictionary containing the image path and its label.

In [ ]:
# Number of unique types
len(np.unique(data_dict['type']))

### Class Map ###

In [ ]:
class_map = {label: x for x, label in enumerate(set(data_dict['type']))}

In [ ]:
class_map

### Custom Dataset ###

In [ ]:
# Attempt to create a custom dataset
class Data(Dataset):
    def __init__(self, data_dict, class_map = class_map, transformations = None):
        self.data_dict = data_dict
        self.class_map = class_map
        self.transformations = transformations

    def __len__(self):
        return(len(self.data_dict['type']))

    def __getitem__(self, idx):
        image = Image.open(self.data_dict['img_path'][idx])
        label = self.class_map[self.data_dict['type'][idx]]
        if self.transformations:
            image = self.transformations(image)
            label = torch.tensor(label)
        return image, label
                

### Some Utility Functions ###

In [ ]:
# Creating seeds to make things fixed
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
# Setting seed and creating my autobot
set_seed(43)
optimus = transforms.Compose([transforms.RandomRotation(5),transforms.RandomHorizontalFlip(0.5),transforms.Resize((80,60)),transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),transforms.ToTensor()])

### Initialization ###

In [ ]:
# Creating my Data
df = Data(data_dict, class_map, transformations = optimus)

In [ ]:
# Splitting the data
train_set,test_set,val_set = random_split(df, [0.8,0.1,0.1])

In [ ]:
# Let's create batches! 
bs = 40

#Data Loaders
train_data = DataLoader(train_set, batch_size= bs, shuffle= True)
test_data = DataLoader(test_set, batch_size= bs, shuffle= True)
val_data = DataLoader(val_set, batch_size= bs, shuffle= False)

In [ ]:
xb,yb = iter(train_data).__next__()

### Visualization ###

In [ ]:
# Time to visualize
vis = {d:f for f,d in df.class_map.items()}
fig, axs = plt.subplots(nrows=2, ncols=5)

c = 0
for row in range(2):
    for col in range(5):
        axs[row, col].imshow(xb[c].permute(1,2,0))
        axs[row, col].axis('off')
        axs[row, col].set_title(vis[yb[c].item()])
        c += 1
plt.show()

# Time for Fashion le Passion! (CNN) #

    We will begin to construct a small CNN model that can take on the task!

In [ ]:
print(xb.shape)

In [ ]:
# class FLP(nn.Module):
#     def __init__(self, num_classes):
#         super(FLP,self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) 
#         self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1) 
#         self.fc1 = nn.Linear(512 * 2 * 1, 256) 
#         self.fc2 = nn.Linear(256, 128)  
#         self.fc3 = nn.Linear(128, num_classes)
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.pool(F.relu(self.conv4(x)))
#         x = self.pool(F.relu(self.conv5(x)))
#         #print(f"Shape after pooling: {x.shape}")
#         x = x.view(x.size(0), -1)
#         #print(f"Shape after view: {x.shape}")
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x)) 
#         x = self.fc3(x) 
#         return x

In [ ]:
# First model
class FLP(nn.Module): 
    def __init__(self, num_classes): 
        super(FLP, self).__init__() 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1) 
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) 
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) 
        self.fc1 = nn.Linear(128 * 10 * 7, 64) 
        self.fc2 = nn.Linear(64, num_classes) 
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x))) 
        x = self.pool(F.relu(self.conv3(x))) 
        # print(f"Shape after pooling: {x.shape}")  
        x = x.view(x.size(0), -1) 
        # print(f"Shape after view: {x.shape}") 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x) 
        return x

In [ ]:
# setting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Ardeur means passion in french
Ardeur = FLP(142)
Ardeur = Ardeur.to(device)

In [ ]:
# setup loss function and optimizers and epoch!
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Ardeur.parameters(), lr=0.0001)
epochs = 30

In [ ]:
# training time! 

for epoch in range(epochs):
    print(f"Starting epoch {epoch + 1}/{epochs}")
    Ardeur.train()
    running_loss = 0.0
    for i, (x,y) in enumerate(train_data):
        x = x.to(device)
        y = y.to(device)
        #print(x.shape)
        optimizer.zero_grad()
        out = Ardeur(x)
        #print(out.shape)
        #print(y.shape)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_data)}], Loss: {loss.item():.4f}")

In [ ]:
Ardeur.eval() 
correct = 0 
total = 0
true_labels = []
pred_labels = []
for epoch in range(epochs):
    with torch.no_grad():
        for x,y in val_data:
            x = x.to(device)
            y = y.to(device)
            out = Ardeur(x)
            _, pred = torch.max(out.data,1)
            #print(pred)
            total += y.size(0)
            correct += (pred == y).sum().item()
            true_labels.extend(y.cpu().numpy())
            pred_labels.extend(pred.cpu().numpy())
    val_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Validation Accuracy: {val_accuracy:.2f}%")
    
true_labels = torch.tensor(true_labels)
pred_labels = torch.tensor(pred_labels)
print("Classification report:")
print(classification_report(true_labels,pred_labels))

In [ ]:
tes = Image.open(data_dict['img_path'][9999])
def predict(model, img):
    img_tens = transforms.ToTensor()(img)
    img_tens = img_tens.to(device)
    plt.imshow(img)
    model.to(device)
    model.eval()
    with torch.no_grad():
        output = model(img_tens.unsqueeze(0))
        _, pred = torch.max(output, 1)
        predicted_class_name = [k for k, v in class_map.items() if v == pred.item()][0]
        print("Predicted class:", predicted_class_name)

In [ ]:
predict(Ardeur,tes)

# Assumptions and Comments #
    The model has an average accuracy of 82% which makes it a highly functioning model. It is a simple model for a somewhat large dataset. Honestly quite impressed. One of my assumptions is that this model can now accurately predict what fashion product is that based on an image. 

# Recommendations # 
    Simply Hyperparameter Tuning and the data itself. This data is only the small version, the actual dataset contains about 140k images that can be useful and might yield a more accurate model. For hyperparameter tuning, it is recommended to tweak certain numbers to achieve varying results.

# Conclusion #
    -> Downloaded the data
    -> Created a data dictionary containing the image paths and their types by matching the img id with the id from the csv file by turning the image folder into a dataframe
    -> Created the classmap based on the dictionary
    -> Created the Custom Dataset
    -> Reused some utility functions
    -> Initialized the data that will be used for training the model
    -> Visualized some samples # Note that there was an error in the visualization, hence we had to debug the data dictionary as data mismatch were observed
    -> Fashion le Ardeur!
    -> Training
    -> Evaluation